In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping

# 데이터 로드 및 기본 전처리
df = pd.read_csv('winemag-data-130k-v2.csv')
df['variety'] = df['variety'].fillna('')
df['description'] = df['description'].fillna('').str.lower()

# 주요 품종 선택 (타겟 클래스)
target_varieties = ['Cabernet Sauvignon', 'Merlot', 'Pinot Noir', 'Chardonnay', 'Riesling', 'Sauvignon Blanc']
df = df[df['variety'].isin(target_varieties)]

# 품종을 정수로 인코딩
df['variety_encoded'] = df['variety'].astype('category').cat.codes
y = pd.get_dummies(df['variety_encoded']).values

# 텍스트 데이터 전처리
max_words = 15000  # 사용할 최대 단어 수
max_len = 120      # 시퀀스 길이

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['description'])
X = tokenizer.texts_to_sequences(df['description'])
X = pad_sequences(X, maxlen=max_len)

# 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 정의
model_multi = Sequential()
model_multi.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model_multi.add(Bidirectional(LSTM(64, return_sequences=True)))
model_multi.add(Dropout(0.5))
model_multi.add(LSTM(64, return_sequences=False))
model_multi.add(Dropout(0.5))
model_multi.add(Dense(64, activation='relu'))
model_multi.add(Dense(len(target_varieties), activation='softmax'))

# 모델 컴파일
model_multi.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 조기 종료 콜백
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# 모델 학습
history = model_multi.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

# 모델 평가
loss, accuracy = model_multi.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# 추가 평가 지표 계산
y_pred = model_multi.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

print(classification_report(y_true_classes, y_pred_classes, target_names=target_varieties))
print("Confusion Matrix:")
print(confusion_matrix(y_true_classes, y_pred_classes))


<ipython-input-11-4e4790c2bfc0>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['variety_encoded'] = df['variety'].astype('category').cat.codes


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


478/478 ━━━━━━━━━━━━━━━━━━━━ 259s 527ms/step - accuracy: 0.5303 - loss: 1.1497 - val_accuracy: 0.8003 - val_loss: 0.5592
Epoch 2/20
478/478 ━━━━━━━━━━━━━━━━━━━━ 266s 537ms/step - accuracy: 0.8332 - loss: 0.4688 - val_accuracy: 0.8274 - val_loss: 0.4748
Epoch 3/20
478/478 ━━━━━━━━━━━━━━━━━━━━ 262s 536ms/step - accuracy: 0.8782 - loss: 0.3475 - val_accuracy: 0.8350 - val_loss: 0.4614
Epoch 4/20
478/478 ━━━━━━━━━━━━━━━━━━━━ 296s 608ms/step - accuracy: 0.9050 - loss: 0.2733 - val_accuracy: 0.8466 - val_loss: 0.4706
Epoch 5/20
478/478 ━━━━━━━━━━━━━━━━━━━━ 292s 547ms/step - accuracy: 0.9228 - loss: 0.2169 - val_accuracy: 0.8448 - val_loss: 0.5127
Epoch 6/20
478/478 ━━━━━━━━━━━━━━━━━━━━ 263s 550ms/step - accuracy: 0.9374 - loss: 0.1829 - val_accuracy: 0.8386 - val_loss: 0.5712
299/299 ━━━━━━━━━━━━━━━━━━━━ 20s 67ms/step - accuracy: 0.8373 - loss: 0.4655
Test Loss: 0.4378267824649811
Test Accuracy: 0.8465082049369812
299/299 ━━━━━━━━━━━━━━━━━━━━ 24s 76ms/step
                    precision    re

In [ ]:
# 6가지 품종에 따른 음식 페어링 사전
food_pairings = {
    'Cabernet Sauvignon': 'Grilled steak, lamb',
    'Merlot': 'Roast chicken, pasta',
    'Pinot Noir': 'Duck, mushroom risotto',
    'Chardonnay': 'Chicken, seafood',
    'Riesling': 'Spicy Asian food, pork',
    'Sauvignon Blanc': 'Salad, goat cheese'
}

# 사용자가 입력한 와인 설명으로 예측
user_description = input("Enter a wine description: ")

# 입력된 텍스트 전처리
user_seq = tokenizer.texts_to_sequences([user_description])
user_seq = pad_sequences(user_seq, maxlen=max_len)

# 와인 품종 예측
wine_type_prediction = model_multi.predict(user_seq)
predicted_variety_index = np.argmax(wine_type_prediction)

# 예측된 와인 품종
predicted_variety = target_varieties[predicted_variety_index]

# 예측된 품종에 따른 음식 페어링 제공
if predicted_variety in food_pairings:
    print(f"Recommended variety: {predicted_variety}")
    print(f"Recommended food pairing: {food_pairings[predicted_variety]}")
else:
    print(f"No pairing found for {predicted_variety}.")


Enter a wine description: Lifted cedar and pine notes interspersed with appealing woodsy scents give way to a surprisingly intense palate that shows some nice peppery notes but ultimately falls short on the finish. Nearly imperceptible tannins suggest early consumption.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
Recommended variety: Pinot Noir
Recommended food pairing: Duck, mushroom risotto


gpt에게네덜란드 배비치 블랙 소비뇽 블랑의 영어로 된 와인 리뷰를 한개만 가져다 줘 질문

Here is an English wine review for the Babich Black Label Sauvignon Blanc 2023:

The wine is described as inviting and musky, with aromas of gooseberry, mandarin, and a hint of fennel on the nose. On the palate, it is fresh and lively with flavors of lemon, lime, gooseberry, and blackcurrant. The mid-palate develops with additional flavors of grapefruit and honeydew melon, while pineapple is noticeable on the finish. The wine is characterized as soft and layered, with a lingering spicy finish

이 와인은 매력적이고 머스크 향이 난다고 묘사되며, 코에서는 구스베리, 만다린, 약간의 회향 향이 느껴집니다. 입안에서는 신선하고 생기 넘치며, 레몬, 라임, 구스베리, 블랙커런트의 풍미가 있습니다. 중간 맛에서는 자몽과 허니듀 멜론의 추가적인 맛이 발전하며, 마무리에서는 파인애플이 두드러집니다. 이 와인은 부드럽고 층이 있는 특징을 가지며, 매운 맛이 오래 지속되는 마무리를 가지고 있습니다.

gpt에게 클라우드베이 피노누아의 영어로 된 와인 리뷰를 한개만 가져다 줘 질문

이 와인은 시더와 소나무의 향이 돋보이며, 매력적인 숲속 향이 어우러져 있습니다. 예상외로 강렬한 맛을 보여주며, 고소한 후추 향도 느껴지지만, 마무리에서는 다소 부족한 느낌을 줍니다. 거의 감지되지 않는 탄닌이 있어, 이 와인은 빨리 소비하는 것이 좋을 것 같습니다.
Lifted cedar and pine notes interspersed with appealing woodsy scents give way to a surprisingly intense palate that shows some nice peppery notes but ultimately falls short on the finish. Nearly imperceptible tannins suggest early consumption.


1. 정확도 (Accuracy): 정의: 전체 데이터 중에서 맞게 예측한 비율입니다.

쉽게 말하면:
100개의 와인을 예측했는데, 87개를 맞췄다면 정확도 = 87%
한계점: 데이터가 불균형(어느 한쪽 데이터가 훨씬 많음)할 때는 신뢰하기 어려울 수 있어요.
예) 90개의 빨간 와인, 10개의 흰 와인이 있으면 전부 빨간 와인이라고 예측해도 90% 정확도가 나올 수 있어요.

2. 정밀도 (Precision): 정의: 모델이 어떤 종류의 와인이라고 예측한 것들 중에서 정말 그 종류인 비율입니다.

쉽게 말하면:
모델이 "이 와인은 빨간 와인!"이라고 예측한 100병 중에서, 진짜 빨간 와인은 몇 병인가요?
왜 중요할까?
예측한 와인이 틀리면 큰 문제가 되는 상황에서 중요해요.
예) "빨간 와인은 고기랑 잘 어울려요"라고 추천했는데 흰 와인이라면 식사 경험이 망할 수 있죠!

3. 재현율 (Recall): 정의: 전체 빨간 와인 중에서 모델이 얼마나 많이 "빨간 와인"이라고 맞췄는지의 비율입니다.

쉽게 말하면:
100개의 빨간 와인 중에서 모델이 "빨간 와인!"이라고 맞춘 병이 몇 병인가요?
왜 중요할까?
모든 빨간 와인을 잘 찾아야 하는 상황에서 중요해요.
예) 어떤 병이 빨간 와인인데 모델이 모르고 흰 와인이라고 하면 문제가 될 수 있죠.

4. F1 점수 (F1-Score): 정의: 정밀도와 재현율을 적절히 조화시킨 평균 점수입니다.

쉽게 말하면:
정확히 맞추는 것도 중요하고, 놓치는 것도 없게 하는 것을 동시에 잘할 때 높은 점수가 나와요.
왜 중요할까?
데이터가 불균형한 경우(빨간 와인이 훨씬 많을 때) 정확도 대신 F1 점수를 확인하면 더 좋은 판단을 할 수 있어요.

5. Support는 각 클래스(예: 빨간 와인, 흰 와인)에 해당하는 실제 데이터의 개수를 의미합니다. 즉, 평가지표를 계산하는 데 사용된 데이터 샘플의 수라고 보면 됩니다.


6. 혼동 행렬 (Confusion Matrix): 정의: 모델이 예측한 것과 실제 값이 어떻게 매칭되는지 보여주는 표입니다.

쉽게 말하면:

"빨간 와인"을 맞췄는지 틀렸는지, "흰 와인"을 맞췄는지 틀렸는지 눈으로 확인 가능합니다.

예시:

    실제\예측	    빨간 와인	    흰 와인

    빨간 와인	        80	         10

    흰 와인	           5              5
빨간 와인 80개를 맞췄고, 10개를 흰 와인으로 틀림.

흰 와인 5개를 맞췄고, 5개를 빨간 와인으로 틀림.

